## Install dependencies

This may take a long time ... ~ 20 mins


In [1]:
%pip install -r requirements.txt

  Using cached tensorflow-2.10.0-cp38-cp38-win_amd64.whl (455.9 MB)
  Using cached tensorflow_gpu-2.10.0-cp38-cp38-win_amd64.whl (455.9 MB)
  Using cached matplotlib-3.6.0-cp38-cp38-win_amd64.whl (7.2 MB)
  Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached h5py-3.7.0-cp38-cp38-win_amd64.whl (2.6 MB)
  Using cached flatbuffers-2.0.7-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.27.0-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached numpy-1.23.3-cp3

You should consider upgrading via the 'c:\Users\Bahillo\Desktop\Simbi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


## Upload data

Group images by separating each class into one folder then wrap all the folder into another folder.

Ex.
```
data/ 
  │
  └─── class1/
  │        │
  |        └─── image1.png
  │        └─── image2.jpg
  |        └─── ...
  │   
  └─── class2/
  │        │
  |        └─── image123.png
  │        └─── image456.jpg
  |        └─── ...
  |
  └─── .../
           │
           └─── ...
```





## Remove unecessary files

Image file extension that are only acceptable are selected in the `image_exts`.

In [5]:
import imghdr, os
import numpy as np

In [6]:
data_dir = './Banana Leaf Disease'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [7]:
nr = 0
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir, image_class)):
    image_path = os.path.join(data_dir, image_class, image)
    try:
      tip = imghdr.what(image_path)
      if tip not in image_exts:
        print(f'Image not in ext list {image_path}')
        os.remove(image_path)
        nr += 1
    except Exception as e:
        print(f'Issue with image {image_path}')

print(f'Removed {nr} unwanted files.')

Removed 0 unwanted files.


## Configure settings

Apply all the settings here for preprocessing, building, and training the neural network.

In [8]:
IMAGE_SIZE = (256, 256) # Square sized are recommended for stability
EPOCHS = 20
BATCH_SIZE = ...
KERNEL_SIZE = (3, 3)
STRIDES = 1

# Ratio for splitting dataset
TRAIN_VAL = 0.7
VALID_VAL = 0.2
TEST_VAL = 0.1

## Prepare, randomize, and normalize the images

Using the `tf.keras.utils.image_dataset_from_directory` with image_size of (256, 256).

See https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [9]:
import tensorflow as tf

In [12]:
data = tf.keras.utils.image_dataset_from_directory(data_dir, image_size=IMAGE_SIZE, shuffle=True) # Images are resized and shuffled
class_names = data.class_names
data = data.map(lambda x, y : (x/255, y)) # Normalize data between 0 and 1

Found 268 files belonging to 4 classes.


In [13]:
for image_batch, labels_batch in data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 256, 256, 3)
(32,)


## Split the input and output pairs for training

Randomly split input and output pairs into sets of data: 70% for training, 20% for validation, and 10% for testing.

  - the training set is used to train the model
  - the validation set is used to measure how well the model is performing during training
  - the testing set is used to test the model after training

In [14]:
train_size = round(len(data)*TRAIN_VAL)
val_size = round(len(data)*VALID_VAL)
test_size = round(len(data)*TEST_VAL)
assert train_size + val_size + test_size == len(data), f'sum must be {len(data)}'

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)


## Configure the dataset for performance

* [Dataset.cache](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

* [Dataset.prefetch](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) overlaps data preprocessing and model execution while training.

In [22]:
AUTOTUNE = tf.data.AUTOTUNE

train = train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)
test = test.cache().prefetch(buffer_size=AUTOTUNE)

## Build & Train the Model

Build and train a [TensorFlow](https://www.tensorflow.org) model using the high-level [Keras](https://www.tensorflow.org/guide/keras) API.

### Build the neural network

In [23]:
num_classes = len(class_names)

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, KERNEL_SIZE, STRIDES, padding='same', activation='relu', input_shape=IMAGE_SIZE+(3,)))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(32, KERNEL_SIZE, STRIDES, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(64, KERNEL_SIZE, STRIDES, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


model.compile('adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 32, 32, 64)      

### Train

In [24]:
history = model.fit(train, epochs=20, validation_data=val)

Epoch 1/20
6/6 [==============================] - 10s 2s/step - loss: 3.1518 - accuracy: 0.3177 - val_loss: 1.2786 - val_accuracy: 0.5000
Epoch 2/20
6/6 [==============================] - 10s 2s/step - loss: 1.1243 - accuracy: 0.4844 - val_loss: 0.9039 - val_accuracy: 0.6094
Epoch 3/20
6/6 [==============================] - 10s 2s/step - loss: 0.8479 - accuracy: 0.6615 - val_loss: 0.7451 - val_accuracy: 0.7031
Epoch 4/20
6/6 [==============================] - 9s 1s/step - loss: 0.6954 - accuracy: 0.7708 - val_loss: 0.6423 - val_accuracy: 0.7656
Epoch 5/20
6/6 [==============================] - 10s 2s/step - loss: 0.4625 - accuracy: 0.8229 - val_loss: 0.6735 - val_accuracy: 0.6719
Epoch 6/20
6/6 [==============================] - 10s 2s/step - loss: 0.4951 - accuracy: 0.8021 - val_loss: 0.5281 - val_accuracy: 0.7656
Epoch 7/20
6/6 [==============================] - 9s 2s/step - loss: 0.3348 - accuracy: 0.8958 - val_loss: 0.3929 - val_accuracy: 0.8281
Epoch 8/20
6/6 [====================

## Evaluate with test data

In [25]:
from tabulate import tabulate

points = test.map(lambda x, y: x)
labels = test.map(lambda x, y: y)
test_yhat = model.predict(points)
test_yhat = tf.math.argmax(test_yhat, -1)

# print the predictions and the expected ouputs
result = list(zip(np.array(test_yhat), list(labels.as_numpy_iterator())[0]))
delta = [True if elem[0] == elem[1] else False for elem in result]
table = list(zip(*zip(*result), delta))

print(tabulate(table, headers=["Predictions", "Expected", "Result"], tablefmt="psql"))

loss, acc = model.evaluate(test)
print(f'Model loss (Test set): {loss}')
print(f'Model Accuracy (Test set): {acc}')

1/1 [==============================] - 0s 335ms/step
+---------------+------------+----------+
|   Predictions |   Expected | Result   |
|---------------+------------+----------|
|             1 |          1 | True     |
|             3 |          3 | True     |
|             3 |          3 | True     |
|             0 |          0 | True     |
|             1 |          1 | True     |
|             0 |          0 | True     |
|             3 |          3 | True     |
|             2 |          0 | False    |
|             3 |          3 | True     |
|             1 |          1 | True     |
|             0 |          0 | True     |
|             1 |          1 | True     |
+---------------+------------+----------+
1/1 [==============================] - 0s 227ms/step - loss: 0.3526 - accuracy: 0.9167
Model loss (Test set): 0.35258087515830994
Model Accuracy (Test set): 0.9166666865348816


## Save the model

 To test different brand new images that are not in the `data_dir`.

### .h5

In [26]:
model.save('./models/model.h5')

### .tflite

In [27]:
# Convert the model to the TensorFlow Lite format

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# # USE QUANTIZATION
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("./models/model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Bahillo\AppData\Local\Temp\tmp2b0magaa\assets


INFO:tensorflow:Assets written to: C:\Users\Bahillo\AppData\Local\Temp\tmp2b0magaa\assets


33655512

#### Optional
For edge devices, see [supported platforms](https://www.tensorflow.org/lite/microcontrollers#supported_platforms)

If the device requires to use C header file (.h) ...





In [28]:
def hex_to_c_array(hex_data) -> str:
    # Declare C variable
    c_str = 'unsigned char model[] = {'
    hex_array = []

    for i, val in enumerate(hex_data):
        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n'
        hex_array.append(hex_str)

    # Wrapping up
    c_str += '\n ' + format(' '.join(hex_array)) + '};'

    return c_str

with open('./models/model.h', 'w') as f:
    content = hex_to_c_array(tflite_model)
    f.write(content)

## Test new images

### Load the model using the **.h5** file

In [29]:
loaded_model =  tf.keras.models.load_model('./models/model.h5')

### Test the result

In [39]:
img = tf.keras.utils.load_img('yellowsigatokatest.jpg', target_size=IMAGE_SIZE) # replace with your file name here
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array/255, 0) # Scale from 0 to 1 and create a batch

yhat = model.predict(img_array)
score = yhat[0]

print(score)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 60ms/step
[2.5089437e-03 3.4435138e-10 2.5526322e-07 9.9749076e-01]
This image most likely belongs to Yellow Sigatoka with a 99.75 percent confidence.
